In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import *
#%run COVID_Visualization_Functions.ipynb
# define fonts for the plots
font = {'family': 'Arial',
        'color':  'black',
        'weight': 'normal',
        'size': 20,
        }

In [3]:
#download most recent data from the cdc website
url = 'https://data.cdc.gov/api/views/9mfq-cb36/rows.csv?accessType=DOWNLOAD'
data = pd.read_csv(url)

#combining NY and NYC data for the map
mapdata = pd.read_csv(url)
mapdata.loc[mapdata['state'] == 'NYC', 'state'] = 'NY'

#creating copy for the last seven days
lastWeekdata = mapdata.copy()

In [4]:
# create dict with the state names and abrv since state name not in data
stateDict = {'US':'United States','AK':'Alaska', 'AL':'Alabama', 'AR':'Arkansas', 'AS':'American Samoa', 'AZ':'Arizona', 'CA':'California', 'CO':'Colorado', 'CT':'Connecticut', 'DC':'District of Columbia', 'DE':'Delaware', 'FL':'Florida', 'FSM':'Federated States of Micronesia', 'GA':'Georgia', 'GU':'Guam', 'HI':'Hawaii', 'IA':'Iowa', 'ID':'Idaho', 'IL':'Illinois', 'IN':'Indiana', 'KS':'Kansas', 'KY':'Kentucky', 'LA':'Louisiana', 'MA':'Massachusetts', 'MD':'Maryland', 'ME':'Maine', 'MI':'Michigan', 'MN':'Minnesota', 'MO':'Missouri', 'MP':'Northern Mariana Islands', 'MS':'Mississippi', 'MT':'Montana', 'NC':'North Carolina', 'ND':'North Dakota', 'NE':'Nebraska', 'NH':'New Hampshire', 'NJ':'New Jersey', 'NM':'New Mexico', 'NV':'Nevada', 'NY':'New York (excludes NYC)', 'NYC':'New York City', 'OH':'Ohio', 'OK':'Oklahoma', 'OR':'Oregon', 'PA':'Pennsylvania', 'PR':'Puerto Rico', 'PW':'Palau', 'RI':'Rhode Island', 'RMI':'Republic of Marshall Islands', 'SC':'South Carolina', 'SD':'South Dakota', 'TN':'Tennessee', 'TX':'Texas', 'UT':'Utah', 'VA':'Virginia', 'VI':'Virgin Islands', 'VT':'Vermont', 'WA':'Washington', 'WI':'Wisconsin', 'WV':'West Virginia', 'WY':'Wyoming'}
namesDF = pd.DataFrame(list(stateDict.items()), columns = ['abv','name'])

#Create a list of all the state names
stateNames = namesDF['name'].values.tolist()

#create list pf names for map to work with
mapNamesDF = pd.DataFrame(list(stateDict.items()), columns = ['abv','name'])
mapNamesDF['pop'] = np.zeros(len(stateNames))

In [5]:
#Create a list of all the state abreviations
stateAbrvs = sorted(pd.DataFrame(data, columns=['state']).drop_duplicates().values.tolist())
stateAbrvs = [ item for elem in stateAbrvs for item in elem]

#Returns state abreviation based on the state name
def stateAbrv(state):
    indx =  namesDF.loc[namesDF['name']==state].index[0]
    abrv = namesDF['abv'][indx]
    return abrv

#returns state name from abreviation
def stateN(abrv):
    indx =  namesDF.loc[namesDF['abv']==abrv].index[0]
    state = namesDF['name'][indx]
    if abrv == 'NY':
        state = 'New York'
    return state

In [6]:
#Create DF with the population data from the census burrow
popUrl = 'http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-popchg2010_2019.csv'
popDF = pd.read_csv(popUrl)

#create list with the states that have population information delete the ones that don't
popNames = popDF['NAME'].values.tolist()
mapNames = mapNamesDF['name'].values.tolist()
for st in stateNames:
    if st in ['New York City','American Samoa','Federated States of Micronesia','Guam','Northern Mariana Islands','Palau','Republic of Marshall Islands','Virgin Islands'] and st in mapNames:
        indx =  mapNamesDF.loc[mapNamesDF['name']==st].index[0]
        mapNamesDF.drop([indx], inplace = True)
    if st in "New York (excludes NYC)" and st in mapNames:
        mapNamesDF.loc[mapNamesDF['name'] == st, 'name'] = 'New York'
# add the population data to the namesDF for all the states that have info
for st in mapNamesDF['name'].values.tolist():
    if st in popNames:
        pop = popDF.loc[popDF['NAME'] == st,'POPESTIMATE2019'].iloc[0]
        mapNamesDF.loc[mapNamesDF['name'] == st, 'pop'] = pop

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap as Basemap
from matplotlib.colors import rgb2hex
from matplotlib.patches import Polygon
from matplotlib import rcParams
from matplotlib import pyplot
from matplotlib.collections import PatchCollection

def heatMap(vtm,d):
    # Lambert Conformal map of lower 48 states.
    rcParams['figure.figsize'] = 22,12
    fig = plt.figure()
    mapAx = fig.add_subplot(111)
    title = 'Concentration of New COVID-19 Cases by State '+ d +' (Past 7 Days)'
    mapAx.set_title(title, fontdict=font)
    m = Basemap(llcrnrlon=-119,llcrnrlat=20,urcrnrlon=-64,urcrnrlat=49,
            projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
    # draw state boundaries.
    # data from U.S Census Bureau
    # https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html
    shp_info = m.readshapefile('cb_2019_us_state_500k','states',drawbounds=True)
    # values to map the colors by
    vals = vtm[0]
    
    # choose a color for each state based on population density.
    colors={}


    statenames=[]
    cmap_reversed = cm.get_cmap('viridis_r')
    cmap = plt.cm.viridis # use  colormap
    patches = []
    vmin = vtm[2]
    if float(vtm[1]) == 0:
        vmax = .001
    else:
        vmax = vtm[1] # set range.
    for shapedict in m.states_info:
        statename = shapedict['NAME']
        # skip DC and Puerto Rico.
        if statename not in ['Puerto Rico','American Samoa','Guam','United States Virgin Islands','Commonwealth of the Northern Mariana Islands']:
            pop = vals[statename]
            # calling colormap with value between 0 and 1 returns
            # rgba value.  Invert color range (hot colors are high
            # population), take sqrt root to spread out colors more.
            colors[statename] = cmap(1.-((abs(pop)-vmin)/(vmax-vmin))**(3/5))[:3]
        statenames.append(statename)
    # cycle through state names, color each one.
    
    #ax = plt.gca() # get current axes instance
    
    # load the shapefile, use the name 'states'
    m.readshapefile('cb_2019_us_state_500k', name='states', drawbounds=True)

    badIslands = [389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400,  385, 386, 387, 388]

    for nshape,seg in enumerate(m.states):
        # skip DC and Puerto Rico.
        if statenames[nshape] not in ['Puerto Rico','American Samoa','Guam','United States Virgin Islands','Commonwealth of the Northern Mariana Islands']:
            color = rgb2hex(colors[statenames[nshape]]) 
            if statenames[nshape]  in ['Alaska', 'Hawaii','District of Columbia']:
                if statenames[nshape] == 'Hawaii' and nshape not in badIslands:
                    seg = list((b[0] + 5300000, b[1]-1550000) for b in seg)
                if statenames[nshape] == 'District of Columbia':
                    seg = list((b[0]*8-4070000*7, b[1]*8-1950000*7 ) for b in seg)
                # Alaska is large. Rescale it.
                elif statenames[nshape] == 'Alaska':
                    seg = list((b[0]*.35 + 1100000, b[1]*.35 - 1300000) for b in seg)
            poly = Polygon(seg,seg,facecolor=color,edgecolor='black',linewidth=1)
            mapAx.add_patch(poly)
            patches.append(poly)
    plt.annotate('D.C', xy=(4595000, 1841000),  xycoords='data',
                    xytext=(4600000, 2050000), textcoords='data',
                    arrowprops=dict(arrowstyle="->"),fontsize = 16)
    plt.annotate('Alaska', xy=(970000, 600000),  xycoords='data',
                    xytext=(753000, 233000), textcoords='data',
                    arrowprops=dict(arrowstyle="->"),fontsize = 16)
    plt.annotate('Hawaii', xy=(1730000, 222000),  xycoords='data',
                    xytext=(2000000, 200000), textcoords='data',
                    arrowprops=dict(arrowstyle="->"),fontsize = 16)
    
    p = PatchCollection(patches, cmap=cmap_reversed)
    p.set_array([])
    cb = fig.colorbar(p, pad = .01,ticks=[ 0.01,.5, .99] )
    cb.ax.set_yticklabels(['Low', 'Medium', 'High'],fontsize = 16)
    fig.patch.set_facecolor('white')
    return fig



In [8]:
def weekTotalCasesRate(x):
    totalweekCasesRateList = {}
    maxCasesRate = 0
    minCasesRate = 10000000
    for item in stateAbrvs:
        sta = stateN(item)
        if sta in popNames:
            popu = float(mapNamesDF.loc[mapNamesDF['name'] == sta,'pop'].iloc[0])
            cases = abs(x[item])
            caseRate = float(cases)/popu*100000
            if caseRate>maxCasesRate:
                maxCasesRate = caseRate
            if caseRate < minCasesRate:
                minCasesRate = caseRate    
            totalweekCasesRateList[sta] =  caseRate
    return totalweekCasesRateList, maxCasesRate, minCasesRate


In [9]:
# find last 7 days reported for the 7 day view
dateList = pd.to_datetime(data['submission_date'])
lastDate = dateList.max()
firstDate = dateList.min()



totRange = pd.date_range(end =lastDate, start = firstDate)
totRange = pd.to_datetime(totRange).astype(str)


Index(['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26',
       '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31',
       ...
       '2020-11-04', '2020-11-05', '2020-11-06', '2020-11-07', '2020-11-08',
       '2020-11-09', '2020-11-10', '2020-11-11', '2020-11-12', '2020-11-13'],
      dtype='object', length=297)

In [10]:
count=0
for day in totRange:
    # find 7 day range
    dRange = pd.date_range(end =day, periods = 7)
    dRange = pd.to_datetime(dRange).astype(str)
    name = str(day)+'_Rate'+'.png'

    # Extracts only the rows from the last 7 days
    lastWeekdata = mapdata.copy()
    lastWeekdata['submission_date'] = pd.to_datetime(lastWeekdata['submission_date']).astype(str)
    lastWeekdata = lastWeekdata[lastWeekdata.submission_date.isin(dRange)]
    # create lists with total cases and rate
    weekCasesdata = lastWeekdata.groupby('state')['new_case'].sum()
    colorData = weekTotalCasesRate(weekCasesdata)
    fig = heatMap(colorData,str(day))
    plt.savefig(name,dpi = 400)
    count+=1
    plt.close()

